In [ ]:
import serial.tools.list_ports

available_ports = list(serial.tools.list_ports.comports())

if not available_ports:
    print("No serial ports found.")
else:
    for port in available_ports:
        print(f"Available Port: {port.device}")


In [ ]:
import serial

# Change the serial port to match your Arduino's port
ser = serial.Serial('COM5', 9600)  # Windows example, on Linux use '/dev/ttyACM0'

def send_command(command):
    ser.write(command.encode())
    response = ser.readline().decode().strip()
    return response

try:
    while True:
        cmd = input("Enter command (C: Check Water, Q: Quit): ")
        if cmd == 'Q':
            break
        elif cmd == 'C':
            response = send_command('C')
            print("Arduino Response:", response)
        else:
            print("Invalid command. Use 'C' or 'Q.")

except KeyboardInterrupt:
    print("Program terminated.")
finally:
    ser.close()


i want image file save to buffer waiting for image classification model

In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException
import cv2
import io
import numpy as np

app = FastAPI()

# Create a global buffer to store the captured image
image_buffer = None

@app.post("/capture")
async def capture_image(file: UploadFile):
    global image_buffer
    # Check if the uploaded file is an image (you can add more validation here).
    if not file.filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        raise HTTPException(status_code=400, detail="Only image files (JPEG or PNG) are supported.")

    # Read the image from the webcam.
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()

    if not ret:
        raise HTTPException(status_code=500, detail="Failed to capture image from the webcam.")

    # Convert the image to bytes and store it in the global buffer
    _, buffer = cv2.imencode(".jpg", frame)
    image_buffer = buffer.tobytes()

    # Save the uploaded file (if needed)
    with open(file.filename, "wb") as f:
        f.write(file.file.read())

    return {"message": "Image captured and stored for classification"}

@app.get("/process_image")
async def process_image():
    global image_buffer

    if image_buffer is not None:
        # Simulate image classification here, you can replace this with your actual model
        # In this example, we're just returning the image as a response
        return {"image": image_buffer}
    else:
        raise HTTPException(status_code=404, detail="No captured image available for processing")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


In [ ]:
import cv2
import io
from PIL import Image

def capture_image():
    # Capture an image from the webcam.
    cap = cv2.VideoCapture(1)
    
    # Set the capture resolution to achieve a widescreen aspect ratio (16:9).
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # Set the width to 1920 pixels
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # Set the height to 1080 pixels

    ret, frame = cap.read()
    cap.release()

    if not ret:
        return None

    # Convert the image to bytes.
    _, buffer = cv2.imencode(".jpg", frame)
    img_bytes = buffer.tobytes()

    return img_bytes

# Capture the image
image_bytes = capture_image()

if image_bytes:
    # Open the image using PIL
    image = Image.open(io.BytesIO(image_bytes))
    
    # Display the image
    image.show()


In [ ]:
import serial

# Change the serial port to match your Arduino's port
ser = serial.Serial('COM5', 9600)  # Windows example, on Linux use '/dev/ttyACM0'


In [ ]:
def send_command(command):
        ser.write(command.encode())
        response = ser.readline().decode().strip()
        return response


In [ ]:
send_command('C')

In [11]:
import cv2
import numpy as np

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("my_model.h5") # Specify the correct file path when loading the model.

In [7]:
def predict_func(img):
    # plt.figure(figsize=(6,4))
    # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.tight_layout()
    img = cv2.resize(img, (224, 224))
    img = np.reshape(img, [-1, 224, 224,3])
    result = np.argmax(model.predict(img))
    if result == 0: print("\033[94m"+"This image -> Not Bottle"+"\033[0m")
    elif result ==1: print("\033[94m"+"This image -> Bottle"+"\033[0m")

In [12]:
test_img = cv2.imread("img\captured_image.jpg")
predict_func(test_img)

1/1 [==============================] - 0s 130ms/step
This image -> Not Bottle
